## **5. PyTorch训练实战——完整训练套路**

下面用 CIFAR 10 数据集来完成一个完整的分类问题。

### **5.1 准备数据集**

In [ ]:
import torch
import torchvision
from torch import nn
from torch.utils.tensorboard import SummaryWriter # tensorflow需要尽早导入
from torch.utils.data import DataLoader

In [8]:
# 准备数据集
train_data = torchvision.datasets.CIFAR10('./data', train=True, 
                                          transform=torchvision.transforms.ToTensor(),
                                          download=False)
test_data = torchvision.datasets.CIFAR10('./data', train=False, 
                                         transform=torchvision.transforms.ToTensor(),
                                         download=False)

# 查看训练数据集、测试数据集分别有多少张图片
print("Train Dataset's Size: {}".format(len(train_data)))
print("Test Dataset's Size: {}".format(len(test_data)))
# len(train_data), len(test_data)

Train Dataset's Size: 50000
Test Dataset's Size: 10000


In [9]:
# 利用 DataLoader 来加载数据集，正确传递数据集对象
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

### **5.2 搭建神经网络**

使用的网络模型如下图所示：

<div style="text-align: center">
    <img src="./image/3-10.png" width="90%"><br>
</div>

In [14]:
# 搭建神经网络
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10), 
        )
    
    def forward(self, x):
        x = self.model(x)
        return x

In [15]:
# 测试网络的正确性
network = NeuralNetwork()
input = torch.ones((64, 3, 32, 32))
output = network(input)
output.shape

torch.Size([64, 10])

### **5.3 利用网络进行训练**

In [17]:
# 创建网络模型
network = NeuralNetwork()
# 损失函数
loss_fn = nn.CrossEntropyLoss()
# 学习率
# lr = 0.01
lr = 1e-2   # 1 × 10^(-2)
# 优化器
optimizer = torch.optim.SGD(network.parameters(), lr=lr)

In [18]:
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10

for i in range(epoch):
    print("----------第 {} 轮训练开始----------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        imgs, targets = data
        outputs = network(imgs)
        loss = loss_fn(outputs, targets)
        
        # 优化器调优
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step += 1
        # 训练步骤逢百的时候才打印
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))

----------第 1 轮训练开始----------
训练次数：100, Loss：2.288135051727295
训练次数：200, Loss：2.285370349884033
训练次数：300, Loss：2.250706195831299
训练次数：400, Loss：2.169644594192505
训练次数：500, Loss：2.0636801719665527
训练次数：600, Loss：2.1239278316497803
训练次数：700, Loss：2.056734085083008
----------第 2 轮训练开始----------
训练次数：800, Loss：1.889098048210144
训练次数：900, Loss：1.9125136137008667
训练次数：1000, Loss：1.857420802116394
训练次数：1100, Loss：1.8069065809249878
训练次数：1200, Loss：1.9366848468780518
训练次数：1300, Loss：1.6871473789215088
训练次数：1400, Loss：1.803779125213623
训练次数：1500, Loss：1.8286163806915283
----------第 3 轮训练开始----------
训练次数：1600, Loss：1.755973219871521
训练次数：1700, Loss：1.52785062789917
训练次数：1800, Loss：1.7954740524291992
训练次数：1900, Loss：1.9818954467773438
训练次数：2000, Loss：1.5561673641204834
训练次数：2100, Loss：1.4396220445632935
训练次数：2200, Loss：1.653775930404663
训练次数：2300, Loss：1.607715129852295
----------第 4 轮训练开始----------
训练次数：2400, Loss：1.5414866209030151
训练次数：2500, Loss：1.5002042055130005
训练次数：2600, Loss：1.584676265

In [ ]:
# item的作用
import torch
a = torch.tensor(5)
print(a)
print(a.item())

tensor(5)
5


### **5.4 评估训练情况**

In [ ]:
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10

for i in range(epoch):
    print("----------第 {} 轮训练开始----------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        imgs, targets = data
        outputs = network(imgs)
        loss = loss_fn(outputs, targets)
        
        # 优化器调优
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step += 1
        # 训练步骤逢百的时候才打印
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))
        # print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))
        
    # 测试步骤开始（每轮训练后都查看在测试数据集上的loss情况）
    total_test_loss = 0
    with torch.no_grad():   # 没有梯度计算，节约内存
        for data in test_dataloader:
            imgs, targets = data
            outputs = network(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss
    print("整体测试集上的loss: {}".format(total_test_loss))

----------第 1 轮训练开始----------
整体测试集上的loss: 873.3963012695312
----------第 2 轮训练开始----------
整体测试集上的loss: 876.9664916992188
----------第 3 轮训练开始----------
整体测试集上的loss: 924.1622314453125
----------第 4 轮训练开始----------
整体测试集上的loss: 733.2106323242188
----------第 5 轮训练开始----------
整体测试集上的loss: 866.1978759765625
----------第 6 轮训练开始----------
整体测试集上的loss: 790.5870361328125
----------第 7 轮训练开始----------
整体测试集上的loss: 696.77197265625
----------第 8 轮训练开始----------
整体测试集上的loss: 696.7969970703125
----------第 9 轮训练开始----------
整体测试集上的loss: 675.2025756835938
----------第 10 轮训练开始----------
整体测试集上的loss: 654.0797729492188


### **5.5 tensorboard查看训练情况**

In [19]:
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10

# 添加tensorboard
writer = SummaryWriter('./log')

for i in range(epoch):
    print("----------第 {} 轮训练开始----------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        imgs, targets = data
        outputs = network(imgs)
        loss = loss_fn(outputs, targets)
        
        # 优化器调优
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step += 1
        # 训练步骤逢百的时候才打印
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))
            writer.add_scalar('Train_Loss', loss.item(), total_train_step)
        # print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))
        # writer.add_scalar('Train_Loss', loss.item(), total_train_step)
        
    # 测试步骤开始（每轮训练后都查看在测试数据集上的loss情况）
    total_test_loss = 0
    with torch.no_grad():   # 没有梯度计算，节约内存
        for data in test_dataloader:
            imgs, targets = data
            outputs = network(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
    print("整体测试集上的loss: {}".format(total_test_loss))
    writer.add_scalar("Test_Loss", total_test_loss, total_test_step)
    total_test_step += 1
    
    # 保存每一轮训练的模型
    torch.save(network, "network_{}.path".format(i))
    
writer.close()

----------第 1 轮训练开始----------
训练次数：100, Loss：1.004499912261963
训练次数：200, Loss：1.3714478015899658
训练次数：300, Loss：1.2910605669021606
训练次数：400, Loss：1.1755789518356323
训练次数：500, Loss：1.2127759456634521
训练次数：600, Loss：1.3611642122268677
训练次数：700, Loss：0.9830293655395508
整体测试集上的loss: 212.3978179693222
----------第 2 轮训练开始----------
训练次数：800, Loss：1.01309335231781
训练次数：900, Loss：1.2425463199615479
训练次数：1000, Loss：1.0553724765777588
训练次数：1100, Loss：0.9625992774963379
训练次数：1200, Loss：1.0058939456939697
训练次数：1300, Loss：0.9937781095504761
训练次数：1400, Loss：1.1345794200897217
训练次数：1500, Loss：1.0009208917617798
整体测试集上的loss: 190.81010353565216
----------第 3 轮训练开始----------
训练次数：1600, Loss：1.0100057125091553
训练次数：1700, Loss：1.1619864702224731
训练次数：1800, Loss：1.0821768045425415
训练次数：1900, Loss：1.051303744316101
训练次数：2000, Loss：0.92399662733078
训练次数：2100, Loss：1.0712974071502686
训练次数：2200, Loss：1.1239312887191772
训练次数：2300, Loss：1.1094300746917725
整体测试集上的loss: 173.9824411869049
----------第 4 轮训练开始-------

### **5.6 查看分类正确率**

In [20]:
import torch

outputs = torch.tensor([[0.1,0.2],
                        [0.05,0.4]])
print(outputs.argmax(0))    # 竖着看，最大值的索引
print(outputs.argmax(1))    # 横着看，最大值的索引

preds = outputs.argmax(0)
targets = torch.tensor([0,1])
print((preds == targets).sum()) # 对应位置相等的个数

tensor([0, 1])
tensor([1, 1])
tensor(2)


In [ ]:
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10

# 添加tensorboard
writer = SummaryWriter('./log')

for i in range(epoch):
    print("----------第 {} 轮训练开始----------".format(i+1))
    
    # 训练步骤开始
    for data in train_dataloader:
        imgs, targets = data
        outputs = network(imgs)
        loss = loss_fn(outputs, targets)
        
        # 优化器调优
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step += 1
        # 训练步骤逢百的时候才打印
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))
            writer.add_scalar('Train_Loss', loss.item(), total_train_step)
        # print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))
        # writer.add_scalar('Train_Loss', loss.item(), total_train_step)
        
    # 测试步骤开始（每轮训练后都查看在测试数据集上的loss情况）
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():   # 没有梯度计算，节约内存
        for data in test_dataloader:
            imgs, targets = data
            outputs = network(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy
    
    print("整体测试集上的loss: {}".format(total_test_loss))
    print("整体测试集上的accuracy: {}".format(total_accuracy/len(test_data)))
    
    writer.add_scalar("Test_Accuracy", total_accuracy/len(test_data), total_test_step)
    writer.add_scalar("Test_Loss", total_test_loss, total_test_step)
    total_test_step += 1
    
    # 保存每一轮训练的模型
    torch.save(network, "./model/network_{}.path".format(i))
    
writer.close()

----------第 1 轮训练开始----------
训练次数：100, Loss：0.5346081852912903
训练次数：200, Loss：0.5090819001197815
训练次数：300, Loss：0.5422479510307312
训练次数：400, Loss：0.7091524004936218
训练次数：500, Loss：0.4985291659832001
训练次数：600, Loss：0.5808225870132446
训练次数：700, Loss：0.44398781657218933
整体测试集上的loss: 407.5290805399418
整体测试集上的accuracy: 0.00026000000070780516
----------第 2 轮训练开始----------
训练次数：800, Loss：0.47275006771087646
训练次数：900, Loss：0.74887615442276
训练次数：1000, Loss：0.6071299314498901
训练次数：1100, Loss：0.6862225532531738
训练次数：1200, Loss：0.7903950214385986
训练次数：1300, Loss：0.4546363353729248
训练次数：1400, Loss：0.6451870203018188
训练次数：1500, Loss：0.5209717750549316
整体测试集上的loss: 376.0288095623255
整体测试集上的accuracy: 0.00026000000070780516
----------第 3 轮训练开始----------
训练次数：1600, Loss：0.32872894406318665
训练次数：1700, Loss：0.2941220700740814
训练次数：1800, Loss：0.2592940330505371
训练次数：1900, Loss：0.44594305753707886
训练次数：2000, Loss：0.29770275950431824
训练次数：2100, Loss：0.7144792079925537
训练次数：2200, Loss：0.5121883749961853
训练次数

### **5.7 特殊层的作用**

在深度学习中，`model.train()`和`model.eval()`是两个非常重要的方法，它们在 PyTorch 框架中用于切换模型的行为模式，确保在不同阶段（训练与评估）中模型能正确地表现。

1. `model.train()`

`model.train()`方法将模型设置为训练模式。可以认为这就像是告诉模型：“现在是练习时间，要尽可能多地学习和适应。” 在训练模式下，模型会激活所有设计用于训练的特性，如：

- **批量归一化（Batch Normalization）**：这一层会在每个训练批次中动态调整数据的均值和标准差，帮助模型更好地学习。
- **Dropout**：这是一种防止模型过拟合的技术，通过随机关闭（即“丢弃”）网络中的部分神经元来减少模型对训练数据的依赖。

2. `model.eval()`

`model.eval()`方法将模型设置为评估模式。这就像是告诉模型：“表演时间到了，需要表现出你的最佳状态。” 在评估模式下，模型会关闭训练时使用的特定层的某些功能：

- **批量归一化**：不再更新数据的统计信息，而是使用训练时已经学习到的均值和标准差。
- **Dropout**：完全关闭，确保所有神经元都参与到模型的运算中，以便模型可以利用其全部能力来进行预测。

这两个模式的切换对于训练一个可靠的模型至关重要。如果在训练阶段不使用`model.train()`，模型可能就无法正确使用Dropout等技术，从而无法有效防止过拟合。同样，如果在评估或测试阶段不使用`model.eval()`，模型的Dropout层仍会随机关闭一部分神经元，导致输出结果不稳定，不能真实反映模型的性能。

In [21]:
# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10

# 添加tensorboard
writer = SummaryWriter('./log')

for i in range(epoch):
    print("----------第 {} 轮训练开始----------".format(i+1))
    
    # 训练步骤开始
    network.train()
    for data in train_dataloader:
        imgs, targets = data
        outputs = network(imgs)
        loss = loss_fn(outputs, targets)
        
        # 优化器调优
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step += 1
        # 训练步骤逢百的时候才打印
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))
            writer.add_scalar('Train_Loss', loss.item(), total_train_step)
        # print("训练次数：{}, Loss：{}".format(total_train_step, loss.item()))
        # writer.add_scalar('Train_Loss', loss.item(), total_train_step)
        
    # 测试步骤开始（每轮训练后都查看在测试数据集上的loss情况）
    network.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():   # 没有梯度计算，节约内存
        for data in test_dataloader:
            imgs, targets = data
            outputs = network(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy += accuracy
    
    print("整体测试集上的loss: {}".format(total_test_loss))
    print("整体测试集上的accuracy: {}".format(total_accuracy/len(test_data)))
    
    writer.add_scalar("Test_Accuracy", total_accuracy/len(test_data), total_test_step)
    writer.add_scalar("Test_Loss", total_test_loss, total_test_step)
    total_test_step += 1
    
    # 保存每一轮训练的模型
    torch.save(network, "./model/network_{}.path".format(i))
    
writer.close()

----------第 1 轮训练开始----------
训练次数：100, Loss：0.8041135668754578
训练次数：200, Loss：0.6040640473365784
训练次数：300, Loss：0.5956678986549377
训练次数：400, Loss：0.739458441734314
训练次数：500, Loss：0.8982022404670715
训练次数：600, Loss：0.8687314391136169
训练次数：700, Loss：0.6455435752868652
整体测试集上的loss: 162.23805141448975
整体测试集上的accuracy: 0.6514000296592712
----------第 2 轮训练开始----------
训练次数：800, Loss：0.7926114797592163
训练次数：900, Loss：0.8763909935951233
训练次数：1000, Loss：1.001367211341858
训练次数：1100, Loss：0.5709820985794067
训练次数：1200, Loss：1.067786455154419
训练次数：1300, Loss：0.7595174312591553
训练次数：1400, Loss：0.7987149953842163
训练次数：1500, Loss：0.6851078867912292
整体测试集上的loss: 166.89498454332352
整体测试集上的accuracy: 0.635699987411499
----------第 3 轮训练开始----------
训练次数：1600, Loss：0.7969492673873901
训练次数：1700, Loss：0.52077317237854
训练次数：1800, Loss：0.7486023902893066
训练次数：1900, Loss：0.7300198674201965
训练次数：2000, Loss：0.8928712010383606
训练次数：2100, Loss：0.644980788230896
训练次数：2200, Loss：1.1257914304733276
训练次数：2300, Loss：0.64